In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Let's start loading the battles for one day

In [ ]:
import csv
csv_filename = "/kaggle/input/clash-royale-season-18-dec-0320-dataset/BattlesStaging_12292020_WL_tagged/BattlesStaging_12292020_WL_tagged.csv"
chunk_size = 500000

text_file_reader = pd.read_csv(csv_filename, engine='python',encoding='utf-8-sig', quoting=csv.QUOTE_MINIMAL, chunksize = chunk_size, index_col = 0)

### Stitch and Load the dataframes into one

In [ ]:
dfList = []
counter = 0

for df in text_file_reader:
    dfList.append(df)
    counter= counter +1
    print("Max rows read: " + str(chunk_size * counter) )

df = pd.concat(dfList,sort=False)

### Let's load the Win Conditions based from this website https://www.deckshop.pro/card/flag/win-condition

In [ ]:
wincon_csv_filename = "/kaggle/input/clash-royale-season-18-dec-0320-dataset/Wincons.csv"
df_wincon = pd.read_csv(wincon_csv_filename, engine='python',encoding='utf-8-sig', quoting=csv.QUOTE_MINIMAL, index_col = 0)

In [ ]:
wincon_dict = dict(zip(df_wincon["card_id"],df_wincon["card_name"]))

### Let's define a function that will populate the wincons of the rows, both for the winner and loser player. We just followed the order of precedence of the wincons here https://www.deckshop.pro/card/flag/win-condition

In [ ]:
import json
def win_con_populator(row):

    winner_win_con_id = 99999999
    loser_win_con_id = 99999999
    
    
    for key, value in wincon_dict.items():
        if((key in json.loads(row['winner.cards.list']))&(winner_win_con_id == 99999999)):
            winner_win_con_id = key
    
        if((key in json.loads(row['loser.cards.list']))&(loser_win_con_id == 99999999)):
            loser_win_con_id = key
            
        if((winner_win_con_id !=99999999)&(loser_win_con_id !=99999999)):
            break
        
    return winner_win_con_id, wincon_dict[winner_win_con_id], loser_win_con_id, wincon_dict[loser_win_con_id]

### Let's populate the wincons

In [ ]:
df['winner.wincon.card.id'] = 99999999
df['loser.wincon.card.id'] = 99999999
df['winner.wincon.card.name'] = None
df['loser.wincon.card.name'] = None
df['winner.wincon.card.id'],df['winner.wincon.card.name'],df['loser.wincon.card.id'],df['loser.wincon.card.name'] = zip(*df.apply(win_con_populator, axis=1))

### Just some eye checks

In [ ]:
df.head()

### Let's define an arena placer function

In [ ]:
def arena_placer(row):
    arena_name = None
    
    if(row['average.startingTrophies'] < 300):
        arena_name = '01.Goblin Stadium'
    elif ((row['average.startingTrophies'] >= 300)&(row['average.startingTrophies'] < 600)):
        arena_name = '02.Bone Pit' 
    elif ((row['average.startingTrophies'] >= 600)&(row['average.startingTrophies'] < 1000)):
        arena_name = '03.Barbarian Bowl'
    #1000
    elif ((row['average.startingTrophies'] >= 1000)&(row['average.startingTrophies'] < 1300)):
        arena_name = '04.P.E.K.K.A\'s Playhouse'
    elif ((row['average.startingTrophies'] >= 1300)&(row['average.startingTrophies'] < 1600)):
        arena_name = '05.Spell Valley'  
    elif ((row['average.startingTrophies'] >= 1600)&(row['average.startingTrophies'] < 2000)):
        arena_name = '06.Builder\'s Workshop'
    # 2000
    elif ((row['average.startingTrophies'] >= 2000)&(row['average.startingTrophies'] < 2300)):
        arena_name = '07.Royal Arena'
    elif ((row['average.startingTrophies'] >= 2300)&(row['average.startingTrophies'] < 2600)):
        arena_name = '08.Frozen Peak'  
    elif ((row['average.startingTrophies'] >= 2600)&(row['average.startingTrophies'] < 3000)):
        arena_name = '09.Jungle Arena'
    # 3000
    elif ((row['average.startingTrophies'] >= 3000)&(row['average.startingTrophies'] < 3300)):
        arena_name = '10.Hog Mountain'
    elif ((row['average.startingTrophies'] >= 3300)&(row['average.startingTrophies'] < 3600)):
        arena_name = '11.Electro Valley'  
    elif ((row['average.startingTrophies'] >= 3600)&(row['average.startingTrophies'] < 4000)):
        arena_name = '12.Spooky Town'
    # 4000
    elif ((row['average.startingTrophies'] >= 4000)&(row['average.startingTrophies'] < 4300)):
        arena_name = '13.Challenger I'
    elif ((row['average.startingTrophies'] >= 4300)&(row['average.startingTrophies'] < 4600)):
        arena_name = '14.Challenger II'
    elif ((row['average.startingTrophies'] >= 4600)&(row['average.startingTrophies'] < 5000)):
        arena_name = '15.Challenger III'
    # 5000
    elif ((row['average.startingTrophies'] >= 5000)&(row['average.startingTrophies'] < 5300)):
        arena_name = '16.Master I'
    elif ((row['average.startingTrophies'] >= 5300)&(row['average.startingTrophies'] < 5600)):
        arena_name = '17.Master II'
    elif ((row['average.startingTrophies'] >= 5600)&(row['average.startingTrophies'] < 6000)):
        arena_name = '18.Master III'
    # 6000
    elif ((row['average.startingTrophies'] >= 6000)&(row['average.startingTrophies'] < 6300)):
        arena_name = '19.Champion'
    elif ((row['average.startingTrophies'] >= 6300)&(row['average.startingTrophies'] < 6600)):
        arena_name = '20.Grand Champion'
    elif ((row['average.startingTrophies'] >= 6600)&(row['average.startingTrophies'] < 7000)):
        arena_name = '21.Royal Champion'         
    elif(row['average.startingTrophies'] >= 7000):
        arena_name = '22.Ultimate Champion' 
        
    return arena_name

df['average.startingTrophies'] = df['average.startingTrophies'].fillna(0)
df['arena.name'] = None
df['arena.name'] = df.apply(arena_placer, axis=1)

### Let's define a plotting function

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

def plot_dataframe(df: pd.DataFrame, yAxisSource, title, xlabel, ylabel, labelField, yscale=None):
    
    ax = df.plot(y=yAxisSource, kind='bar', grid=True, figsize=(24,10), color=['#86bf91'], zorder=2, rot=45)
    
    ax.set_title(title)
    ax.set_xlabel(xlabel, labelpad=20, weight='bold', size=12)
    ax.set_ylabel(ylabel, labelpad=20, weight='bold', size=12)
    
    if(yscale!=None):
        ax.set_yscale(yscale)
    
    # Format y-axis label
    ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,g}'))
    
    rects = ax.patches
    
    # Make some labels.
    labels = df[labelField].tolist()
    
    for rect, label in zip(rects, labels):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width() / 2, height + 5, label,
                ha='center', va='bottom')

    
    plt.show()
    

### Let's get the Matches Per Arena

In [ ]:
df_arena_counts = df.groupby(['arena.name'])[["arena.id"]].count()
df_arena_counts.rename(columns={"arena.id": "count"}, inplace=True)
df_arena_counts.index = [x.split(".",1)[1] for x in df_arena_counts.index]
plot_dataframe(df_arena_counts, ["count"], "Matches Per Arena", "Arena Names", "Matches", 'count', 'log')

### Let's get the Matches Per Champion Arena

In [ ]:
df_arena_counts = df[df["arena.name"].str.contains("Champion", na=False)].groupby(['arena.name'])[["arena.id"]].count()
df_arena_counts.rename(columns={"arena.id": "count"}, inplace=True)
df_arena_counts.index = [x.split(".",1)[1] for x in df_arena_counts.index]
plot_dataframe(df_arena_counts,["count"], "Matches Per Champion Arena", "Arena Names", "Matches", 'count', 'log')

### Let's plot the Win Cons Of the Winners of Matches In The Champions Arenas

In [ ]:
df_winner_win_cons = df[(df["arena.name"].str.contains("Champion", na=False))&(df['winner.wincon.card.name'] != df['loser.wincon.card.name'])].groupby(['winner.wincon.card.name'])["arena.id"].count().reset_index(name='count').sort_values(['count'], ascending=False)
df_loser_win_cons = df[(df["arena.name"].str.contains("Champion", na=False))&(df['winner.wincon.card.name'] != df['loser.wincon.card.name'])].groupby(['loser.wincon.card.name'])["arena.id"].count().reset_index(name='count').sort_values(['count'], ascending=False)
df_winner_win_cons.reset_index(drop=True, inplace=True)
df_loser_win_cons.reset_index(drop=True, inplace=True)

result = df_winner_win_cons.merge(df_loser_win_cons, left_on='winner.wincon.card.name', right_on='loser.wincon.card.name', how = "inner")
result.drop(['loser.wincon.card.name'], axis=1, errors='ignore', inplace=True)
result.rename(columns={"count_x": "win_count","count_y": "lose_count"}, inplace=True)
result['win_percentage'] = result["win_count"] / (result["win_count"]+result["lose_count"])
result['lose_percentage'] = result["lose_count"] / (result["win_count"]+result["lose_count"])
result['total_matches'] = (result["win_count"]+result["lose_count"])
result.set_index("winner.wincon.card.name",inplace=True)

plot_dataframe(result,["win_count"], "Matches Won Per WinCon In Champion Arenas", "Win Cons", "Matches", "win_count","log")